In [47]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [48]:
## load the trained model, scalar, OHE pickle files

model = load_model('model.h5')

with open('label_encoder_gen.pkl', 'rb') as file:
    label_encoder_gen = pickle.load(file)

with open('OHE_Geography.pkl', 'rb' ) as file:
    OHE_Geography = pickle.load(file)
    
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [49]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [50]:
print(type(input_data))
print(input_data['Gender'])

<class 'dict'>
Male


In [51]:
input_data_df = pd.DataFrame([input_data])

In [52]:
# OHE_Geography encoder
geo_encoded = OHE_Geography.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=OHE_Geography.get_feature_names_out(['Geography']))
geo_encoded_df


/Users/rekhajaggannagari/Documents/GenAI/ANN project Implementation/venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [53]:
input_data_df = pd.concat([input_data_df.drop(['Geography'], axis=1), geo_encoded_df], axis=1)

In [54]:
input_data_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [55]:
# label encoder gen
input_data_df['Gender'] = label_encoder_gen.transform(input_data_df['Gender'])
input_data_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [56]:
input_data_df = scaler.transform(input_data_df)

In [57]:
model_prediction = model.predict(input_data_df)

1/1 [==============================] - 0s 22ms/step


In [58]:
model_prediction

array([[0.05652107]], dtype=float32)

In [59]:
prediction_proba = model_prediction[0][0]

In [60]:
if prediction_proba > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.
